In [1]:
import os
import pytz

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from datetime import datetime, timezone
from tqdm.notebook import tqdm

In [2]:
columns = ["Open time", "Open", "High", "Low", "Close", "Volume", "Close time", "Quote asset volume", "n(Trades)", "Taker buy base asset volume", "Taker buy quote asset volume", "Ignore"]

In [3]:
btc_dirs_1d = glob(os.getcwd()+"\\Data\\01 - BTCUSDT\\1d\\*.csv")
eth_dirs_1d = glob(os.getcwd()+"\\Data\\02 - ETHUSDT\\1d\\*.csv")
bnb_dirs_1d = glob(os.getcwd()+"\\Data\\03 - BNBUSDT\\1d\\*.csv")
sol_dirs_1d = glob(os.getcwd()+"\\Data\\04 - SOLUSDT\\1d\\*.csv")
ada_dirs_1d = glob(os.getcwd()+"\\Data\\05 - ADAUSDT\\1d\\*.csv")

btc_usdt_1d = list()
for file in tqdm(btc_dirs_1d):
    data = pd.read_csv(file, header=None)
    for line in data.values.tolist():
        btc_usdt_1d.append(line)
btc_usdt_1d = pd.DataFrame(btc_usdt_1d, columns=columns)
btc_usdt_1d = btc_usdt_1d.loc[btc_usdt_1d["Ignore"]==0, ["Open time", "Open", "High", "Low", "Close", "Volume", "n(Trades)"]]
btc_usdt_1d = btc_usdt_1d.reset_index(drop=True)

eth_usdt_1d = list()
for file in tqdm(eth_dirs_1d):
    data = pd.read_csv(file, header=None)
    for line in data.values.tolist():
        eth_usdt_1d.append(line)
eth_usdt_1d = pd.DataFrame(eth_usdt_1d, columns=columns)
eth_usdt_1d = eth_usdt_1d.loc[eth_usdt_1d["Ignore"]==0, ["Open time", "Open", "High", "Low", "Close", "Volume", "n(Trades)"]]
eth_usdt_1d = eth_usdt_1d.reset_index(drop=True)

bnb_usdt_1d = list()
for file in tqdm(bnb_dirs_1d):
    data = pd.read_csv(file, header=None)
    for line in data.values.tolist():
        bnb_usdt_1d.append(line)
bnb_usdt_1d = pd.DataFrame(bnb_usdt_1d, columns=columns)
bnb_usdt_1d = bnb_usdt_1d.loc[bnb_usdt_1d["Ignore"]==0, ["Open time", "Open", "High", "Low", "Close", "Volume", "n(Trades)"]]
bnb_usdt_1d = bnb_usdt_1d.reset_index(drop=True)

sol_usdt_1d = list()
for file in tqdm(sol_dirs_1d):
    data = pd.read_csv(file, header=None)
    for line in data.values.tolist():
        sol_usdt_1d.append(line)
sol_usdt_1d = pd.DataFrame(sol_usdt_1d, columns=columns)
sol_usdt_1d = sol_usdt_1d.loc[sol_usdt_1d["Ignore"]==0, ["Open time", "Open", "High", "Low", "Close", "Volume", "n(Trades)"]]
sol_usdt_1d = sol_usdt_1d.reset_index(drop=True)

ada_usdt_1d = list()
for file in tqdm(ada_dirs_1d):
    data = pd.read_csv(file, header=None)
    for line in data.values.tolist():
        ada_usdt_1d.append(line)
ada_usdt_1d = pd.DataFrame(ada_usdt_1d, columns=columns)
ada_usdt_1d = ada_usdt_1d.loc[ada_usdt_1d["Ignore"]==0, ["Open time", "Open", "High", "Low", "Close", "Volume", "n(Trades)"]]
ada_usdt_1d = ada_usdt_1d.reset_index(drop=True)

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

In [4]:
data_1d = [btc_usdt_1d, eth_usdt_1d, bnb_usdt_1d, sol_usdt_1d, ada_usdt_1d]

In [5]:
def featureEngineering(df):
    
    def _rsi(df, periods=14, ema=True):
        close_delta = df["Close"].diff()
        up = close_delta.clip(lower=0)
        down = -close_delta.clip(upper=0)
        ma_up = up.ewm(com=periods-1, adjust=True, min_periods=periods).mean()
        ma_down = down.ewm(com=periods-1, adjust=True, min_periods=periods).mean()
        rsi = ma_up / ma_down
        rsi = 100 - (100/(1+rsi))
        return rsi
    
    def _cci(df, periods=20):
        cci = pd.DataFrame(np.mean([df["High"], df["Low"], df["Close"]], axis=0), columns=["TP"])
        cci["MA"] = cci["TP"].rolling(periods).mean()
        cci["MAD"] = cci["TP"].rolling(periods).apply(lambda x: pd.Series(x).mad())
        cci["CCI"] = (cci["TP"]-cci["MA"]) / (0.015*cci["MAD"])
        return cci["CCI"]

    df["MA(7)"] = df["Close"].rolling(7).mean()
    df["MA(25)"] = df["Close"].rolling(25).mean()
    df["MA(99)"] = df["Close"].rolling(99).mean()
    
    df["Change"] = (df["Close"] - df["Open"]) / df["Open"]
    df["Amplitude"] = (df["High"] - df["Low"]) / df["Open"]
    
    df["RSI"] = _rsi(df)
    df["CCI"] = _cci(df)

In [6]:
for df in tqdm(data_1d):
    featureEngineering(df)

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
data_1d[0].to_csv(os.getcwd()+"\\Data\\00 - Processed\\1d\\BTCUSDT-1d-010817-311221.csv", index=False)
data_1d[1].to_csv(os.getcwd()+"\\Data\\00 - Processed\\1d\\ETHUSDT-1d-010817-311221.csv", index=False)
data_1d[2].to_csv(os.getcwd()+"\\Data\\00 - Processed\\1d\\BNBUSDT-1d-011117-311221.csv", index=False)
data_1d[3].to_csv(os.getcwd()+"\\Data\\00 - Processed\\1d\\SOLUSDT-1d-010820-311221.csv", index=False)
data_1d[4].to_csv(os.getcwd()+"\\Data\\00 - Processed\\1d\\ADAUSDT-1d-010518-311221.csv", index=False)